In [ ]:
import sys

import pandas as pd

plot_data_dbscan = pd.read_csv("plotData_bert_dbscan_eps2_samples4.csv")

sys.path.insert(1, '../utils/')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import plotly.io
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import random
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in plot_data_dbscan.db_cluster.unique():
#     if c== -1:
#         trace = go.Scatter(
#             x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
#             y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
#             mode = 'markers',
#             name = str(c),
#             marker = go.scatter.Marker(size=4, color='grey'),
#             text = plot_data_dbscan.index,
#             showlegend = True,
#         )
#     else:
   # Create a trace
    trace = go.Scatter(
        x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
        y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
        mode = 'markers',
        name = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        showlegend = True,
    )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
sys.path.insert(1, '../bert-tfidf/plots/')
iplot(data, filename='dbscan_context_10',image='png', image_height = 500, image_width = 700) 

## Lasso selection with printing elements in a qgrid table

In [ ]:
import time
import qgrid
qgrid.set_grid_option('maxVisibleRows', 5)

import pdb

f = go.FigureWidget()
f.layout.dragmode = 'select'
DISPLAYED_TABLE_COLUMNS = ["left_token_str", "right_token_str", "del_tokens_str", "ins_tokens_str", "db_cluster"]

scatter = f.add_scatter(x = plot_data_dbscan["t-SNE-X"], y = plot_data_dbscan["t-SNE-Y"], mode = 'markers', 
                       marker=go.scatter.Marker(size=0, opacity=0),showlegend = False)


r = lambda: random.randint(0,255)

# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(  
    header=dict(values=DISPLAYED_TABLE_COLUMNS,             
                
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    
    cells=dict(values=[plot_data_dbscan[col] for col in DISPLAYED_TABLE_COLUMNS],              
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5
               ))])



def selection_fn(trace,points,selector):
    global sel_points, counter
    sel_points = points

    #with out:
        #clear_output()
        #display("Finished: " + str(len(points.point_inds)) + "points selected")
    #    display(qgrid.show_grid(plot_data_dbscan.loc[points.point_inds, DISPLAYED_TABLE_COLUMNS]))
    
    t.data[0].cells.values = [plot_data_dbscan.loc[points.point_inds][col] for col in DISPLAYED_TABLE_COLUMNS]


traces = []
for c in plot_data_dbscan["db_cluster"].unique():
    # Create a trace
    trace = go.Scatter(
        x=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-X"],
        y=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-Y"],
        mode = 'markers',
        name = str(c),
        uid = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        
        showlegend = True, 

    )
    
    f.add_trace(trace)
#trace.on_selection(selection_fn)

from ipywidgets import VBox    
scatter.on_selection(selection_fn)
VBox((f, t))


## Nationality

In [ ]:
r = lambda: random.randint(0,255)

traces = []
f.layout.dragmode = 'select'
plot_data_dbscan['opacity'] = 1
plot_data_dbscan.loc[plot_data_dbscan['nationality'].isna(), 'opacity'] = 0.1
for c in plot_data_dbscan.db_cluster.unique():

    if c!=-1:
   # Create a trace
        trace = go.Scatter(
            x=plot_data_dbscan.loc[plot_data_dbscan["db_cluster"]==c, "t-SNE-X"],
            y=plot_data_dbscan.loc[plot_data_dbscan["db_cluster"]==c, "t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r()),opacity=plot_data_dbscan.loc[plot_data_dbscan["db_cluster"]==c, "opacity"]),
            showlegend = True
        )
    traces.append(trace)
data = traces
layout = go.Layout(dragmode = 'select')

fig = go.Figure(data=data, layout=layout)



# Plot and embed in ipython notebook
iplot(fig, filename='basic-scatter')

## Cluster size frequency

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.xlabel('Cluster')
plt.ylabel('Size of a cluster')
plot_data_dbscan['db_cluster'].value_counts().nlargest(n=10).plot(kind='bar')

In [ ]:
plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == 65, ['from_rev','nationality','left_token_str', 'ins_tokens_str', 'right_token_str', 'dels_tokens_str']]

## Token frequency in left and right context

In [ ]:
import operator
import numpy as np
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'medium',
         'axes.titlesize':'medium',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

top = 19

from collections import Counter
right_context_freq = Counter(list(plot_data_dbscan['right_token_str']))
left_context_freq = Counter(list(plot_data_dbscan['left_token_str']))
ins_context_freq = Counter(list(plot_data_dbscan['ins_tokens_str']))
dels_context_freq = Counter(list(plot_data_dbscan['del_tokens_str']))
gap_context_freq = Counter(list(plot_data_dbscan['del_tokens_str']) + list(plot_data_dbscan['ins_tokens_str']))

def plot_freq(left_context_freq,gap_context_freq,  right_context_freq, top, width, height):
    fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(width, height))
        
    axs[0].barh([x for x,_ in left_context_freq],[x for _,x in left_context_freq])
    axs[0].set_title(" frequency plot of {} tokens in left context".format(top))
    axs[0].set_xlabel("frequency")
    
    axs[1].barh([x for x,_ in gap_context_freq ],[x for _,x in gap_context_freq])
    axs[1].set_title(" frequency plot of {} tokens in gap".format(top))
    axs[1].set_xlabel("frequency")
        
    axs[2].barh([x for x,_ in right_context_freq ],[x for _,x in right_context_freq])
    axs[2].set_title(" frequency plot of {} tokens in right context".format(top))
    axs[2].set_xlabel("frequency")
    return fig


fig = plot_freq( [i for i in left_context_freq.most_common(top) if np.nan not in i],  
                [i for i in gap_context_freq.most_common(top) if np.nan not in i],
                [i for i in right_context_freq.most_common(top) if np.nan not in i], top, 20, 10)

In [ ]:
DISPLAYED_TABLE_COLUMNS.remove("db_cluster")


## Most frequent tokens in a cluster

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output
print('Type a cluster id:')
input_cluster = widgets.Text()
display(input_cluster)
top = 10

def handle_submit(sender):
    clear_output()
    cluster = int(input_cluster.value)
    cluster_plot_data = plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == cluster]
    right_context_cluster = Counter(cluster_plot_data['right_token_str'].str.cat(sep=' ').split())
    left_context_cluster = Counter(cluster_plot_data['left_token_str'].str.cat(sep=' ').split())
    ins_cluster = Counter(cluster_plot_data['ins_tokens_str'].str.cat(sep=' ').split())
    del_cluster = Counter(cluster_plot_data['del_tokens_str'].str.cat(sep=' ').split())
    gap_cluster = Counter(cluster_plot_data['ins_tokens_str'].str.cat(sep=' ').split()+cluster_plot_data['del_tokens_str'].str.cat(sep=' ').split())
    fig = plot_freq( [i for i in left_context_cluster.most_common(top) if np.nan not in i],  
                [i for i in gap_cluster.most_common(top) if np.nan not in i], 
                [i for i in right_context_cluster.most_common(top) if np.nan not in i], len(cluster_plot_data), 20, 10)
    
    print('Type a cluster id:')
    display(input_cluster)
    display(qgrid.show_grid(cluster_plot_data[DISPLAYED_TABLE_COLUMNS]))
    
input_cluster.on_submit(handle_submit)


In [ ]:
from plotly.tools import make_subplots
f = go.FigureWidget()
f.layout.dragmode = 'select'

scatter = f.add_scatter(x = plot_data_dbscan["t-SNE-X"], y = plot_data_dbscan["t-SNE-Y"], mode = 'markers', 
                       marker=go.scatter.Marker(size=0, opacity=0),showlegend = False)
left_context_cluster = Counter(plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == 1]['left_token_str'].str.cat(sep=' ').split())
left_context_freq = [i for i in left_context_cluster.most_common() if np.nan not in i]
right_context_cluster = Counter(plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == 1]['right_token_str'].str.cat(sep=' ').split())
right_context_freq = [i for i in right_context_cluster.most_common() if np.nan not in i]
gap_merged = pd.concat([plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == 1]['ins_tokens_str'], plot_data_dbscan.loc[plot_data_dbscan['db_cluster'] == 1]['del_tokens_str']])
gap_context_cluster = Counter(gap_merged.str.cat(sep=' ').split())
gap_context_freq = [i for i in gap_context_cluster.most_common() if np.nan not in i]

fig = make_subplots(rows=1, cols=3)
f2 = go.FigureWidget(fig)
left = f2.add_trace(
    go.Bar(
            x=[x for x,_ in left_context_freq],
            y=[x for _,x in left_context_freq], name="Left context"
    ), row=1, col=1
)

gap = f2.add_trace(
    go.Bar(
            x=[x for x,_ in gap_context_freq],
            y=[x for _,x in gap_context_freq], name="Gap"
    ), row=1, col=2
)

right = f2.add_trace(
    go.Bar(
            x=[x for x,_ in right_context_freq],
            y=[x for _,x in right_context_freq], name="Right context"
    ), row=1, col=3
)

r = lambda: random.randint(0,255)
    
def bar_update(context_column, box):
    context_cluster = Counter(context_column.str.cat(sep=' ').split())
    context_freq = [i for i in context_cluster.most_common(10) if np.nan not in i]
    box.x = [x for x,_ in context_freq]
    box.y = [x for _,x in context_freq]

def selection_fn(trace,points,selector):
    gap_merged = pd.concat([plot_data_dbscan.loc[points.point_inds]['ins_tokens_str'], plot_data_dbscan.loc[points.point_inds]['del_tokens_str']])
    gap_context_cluster = Counter(gap_merged.str.cat(sep=' ').split())
    bar_update(plot_data_dbscan.loc[points.point_inds]['left_token_str'], left)
    bar_update(gap_merged, gap)
    bar_update(plot_data_dbscan.loc[points.point_inds]['right_token_str'], right)

traces = []
for c in plot_data_dbscan["db_cluster"].unique():
    # Create a trace
    trace = go.Scatter(
        x=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-X"],
        y=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-Y"],
        mode = 'markers',
        name = str(c),
        uid = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        text = plot_data_dbscan.db_cluster,
        
        showlegend = True, 

    )
    
    f.add_trace(trace)

from ipywidgets import VBox    
scatter.on_selection(selection_fn)

VBox([f, f2])